In [ ]:
import pandas as pd
import json
import seaborn as sns
from scipy import interp
import numpy
import os
import matplotlib.pyplot as plt
import numpy as np
from pandas import DataFrame

In [ ]:
# 读取所有的json文件名
file_path = '../train_data/'

FilePath_list = []
for root, dirs, files in os.walk(file_path):
    for file in files:
        FilePath    = os.path.join(root, file)
        FilePath_list.append(FilePath)

In [ ]:
clinical_feature_name_file = 'clinical_features_synonym.xlsx'
df_cf = pd.read_excel(clinical_feature_name_file)


clinical_feature_list = []
for i in df_cf["临床特征"].values.tolist():
    clinical_feature_list.append(i)


# 首先获得整个列名
# 也就是把所有变量名称全都加上属性“发热_存在情况”的形式

ssu_attributes = ['存在情况', '严重程度', '持续时间', '痛觉性质', '发作模式', '加剧因素', '缓解因素', '分布模式', '偏侧性', '四分区']

disease_symptom_attr_list = []


# 构建加上属性的表头
for i in clinical_feature_list:
    for ssu_attribute in ssu_attributes:
        disease_symptom_attr = i + '_' + ssu_attribute
        disease_symptom_attr_list.append(disease_symptom_attr)


In [ ]:
# 构建临床特征的同义词字典
term_synonym_list = []
for synonym in df_cf["同义词"].values.tolist():
    term_synonym = synonym.split('||')
    term_synonym_list.append(term_synonym)
term_synonym_dict = dict(zip(clinical_feature_list, term_synonym_list))

In [ ]:
# 构建“疾病名称”的归一化字典
disease_classification_file = 'disease_classification_name.xlsx'
df_classification = pd.read_excel(disease_classification_file)
# df_classification = pd.read_excel(disease_classification_file,sheet_name='慢阻肺')

Classification_group_list = []
for i in df_classification["Classification group"].values.tolist():
    Classification_group_list.append(i)
# print(Classification_group_list)



disease_classification_list = []
for synonym in df_classification["disease_name"].values.tolist():
    disease_synonym = synonym.split('||')
    disease_classification_list.append(disease_synonym)
disease_classification_dict = dict(zip(Classification_group_list, disease_classification_list))

In [ ]:
disease_symptom_clinical_all = []
disease_Inpatient_number_all = []
for file_name in FilePath_list:
    with open(file_name,'r',encoding='utf-8') as load_f:
        term_ann_list =  []
        disease_name = file_name.split('train_data/')[1].split('_')[0]
        Inpatient_number = file_name.split('train_data/')[1].split('_')[1].split('.json')[0]
        disease_Inpatient_number = disease_name + '_' + Inpatient_number
        disease_Inpatient_number_all.append(disease_Inpatient_number)
        for key_d,value_d in disease_classification_dict.items():
            if disease_name in value_d:
                load_dict = json.load(load_f)
                annot_infos = load_dict['标注信息']
                for annot_info in annot_infos.values():
                    term_ann_type = annot_info[0]['术语类型']
                    for ssu_attribute in ssu_attributes:
                        if term_ann_type in ['症状','疾病','影像学表现'] and annot_info[1]['存在情况']['标准取值'] != '否认':
                            term_ann_notlab = annot_info[0]['术语名称']
                            term_attr = annot_info[1][ssu_attribute]['标准取值']
                            for key_s,value_s in term_synonym_dict.items():
                                if term_ann_notlab in value_s:
                                    term_ann_list.append([key_s+'_'+ssu_attribute,term_attr])


                        if term_ann_type == '定量定性检查':
                            term_ann_lab =  annot_info[0]['术语名称'] + annot_info[1]['结果判读']['标准取值']
                            term_attr_lab = annot_info[1][ssu_attribute]['标准取值']
                            for key_s,value_s in term_synonym_dict.items():
                                if term_ann_lab in value_s:
                                    term_ann_list.append([key_s+'_'+ssu_attribute,term_attr_lab])

                        if term_ann_type in ['药物','手术'] and annot_info[1]['存在情况']['标准取值'] != '否认':
                            term_ann_treat =  annot_info[0]['术语名称']
                            term_treat_attr = annot_info[1]['存在情况']['标准取值']
                            for key_s,value_s in term_synonym_dict.items():
                                if term_ann_treat in value_s:
                                    term_ann_list.append([key_s+'_'+'存在情况',term_treat_attr])


                list_fir_symptom = []
                term_ann_first_dict = {}
                for i in range(len(term_ann_list)):
                    if term_ann_list[i][0] not in list_fir_symptom:
                        # print(term_ann_list[i])
                        list_fir_symptom.append(term_ann_list[i][0])
                        term_ann_first_dict[term_ann_list[i][0]] = term_ann_list[i][1]
                # print(term_ann_first_dict)

                res = []
                for x in disease_symptom_attr_list:
                    if x in term_ann_first_dict.keys():
                        res.append(term_ann_first_dict[x])
                    else:
                        res.append('NULL')
                disease_symptom_clinical_all.append([key_d,res])

In [ ]:
term_ann_list = []
for file_name in FilePath_list:
    # print(file_name)
    with open(file_name,'r',encoding='utf-8') as load_f:
        load_dict = json.load(load_f)
        annot_infos = load_dict['标注信息']
        for annot_info in annot_infos.values():
            term_ann_type = annot_info[0]['术语类型']
            if term_ann_type in ['症状','疾病','影像学表现'] and annot_info[1]['存在情况']['标准取值'] != '否认':
                term_ann = annot_info[0]['术语名称']
                term_loca = annot_info[1]['发作部位']
                # 没有部位的直接把术语写进去
                term_ann_list.append(term_ann)
            if term_ann_type == '定量定性检查':
                term_ann_labtest = annot_info[0]['术语名称'] + annot_info[1]['结果判读']['标准取值']
                term_ann_list.append(term_ann_labtest)
                
            if term_ann_type in ['药物','手术'] and annot_info[1]['存在情况']['标准取值'] != '否认':
                term_ann_treat =  annot_info[0]['术语名称']
                term_ann_list.append(term_ann_treat)

In [ ]:
patient_age = []
patient_gender = []
for file_name in FilePath_list:
    with open(file_name,'r',encoding='utf-8') as load_f:
        load_dict = json.load(load_f)
        patient_age_gender =load_dict['原文信息'].split('。')[0].split('岁,')

        patient_age.append(int(patient_age_gender[0]))
        patient_gender.append(patient_age_gender[1].replace('男','1').replace('女','0'))

In [ ]:
df_list = []
disease_name = []
disease_name_1 = []
for disease_symptom_list_ in disease_symptom_clinical_all:
    df_list.append(disease_symptom_list_[1])
    disease_name_ = disease_symptom_list_[0]
    # 疾病名称增加到最后一列
    disease_name_1.append(disease_symptom_list_[0])
    disease_name.append(disease_name_)
    # 住院号增加到后面
data = DataFrame(df_list, columns=disease_symptom_attr_list)


#插入“疾病”“年龄”和“性别”特征
data['诊断名称'] = disease_name
data.insert(loc=0, column='性别', value=patient_gender)
data.insert(loc=1, column='年龄', value=patient_age)

In [ ]:
# 将时间标准化到天
ori_duration = []
std_duration = []
for column in data:
    if '持续时间' in column:
        for duration in data[column]:
            if duration != 'NULL':
                if '年' in duration:
                    std_dura = int(duration.rstrip('年'))*365
                if '月' in duration:
                    std_dura = int(duration.rstrip('月'))*30
                if '周' in duration:
                    std_dura = int(duration.rstrip('周'))*7
                if '天' in duration:
                    std_dura = int(duration.rstrip('天'))
                if '小时' in duration:
                    std_dura = int(duration.rstrip('小时'))/24
                if '分钟' in duration:
                    std_dura = int(duration.rstrip('分钟'))/1440
                ori_duration.append(duration)
                std_duration.append(std_dura)

In [ ]:
# 构建一个需要替换持续时间的字典
std_duration_dict = {}
std_duration_dict = dict(zip(ori_duration, std_duration))

In [ ]:
# 把所有的持续时间全部标准化
data.replace(std_duration_dict,inplace=True)

In [ ]:
# 继续替换，NULL和“否认”全部替换为0
data = data.replace('存在', '1').replace('否认', '0').replace('NULL', '0').replace('既往', '2').replace('可能', '4').replace('假设', '5').replace('家族', '6').replace('轻度', 1).replace('中度', 2).replace('重度', 3).replace('危重', 4).replace('牵拉样痛', '1').replace('针刺样痛', '2').replace('烧灼样痛', '3').replace('钝痛', '4').replace('绞痛', '5').replace('压榨样痛', '6').replace('痉挛样痛', '6').replace('闷痛', '7').replace('急性', '1').replace('亚急性', '1.5').replace('慢性', '2').replace('持续性', '3').replace('反复发作', '4').replace('周期性', '5').replace('间歇性', '6').replace('一过性', '7').replace('迁延不愈', '8').replace('转移性', '9').replace('晨间发作', '10').replace('夜间发作', '11').replace('午后发作', '12').replace('偶然发作', '13').replace('疾病复发', '14').replace('休息后缓解', '1').replace('活动后加剧', '2').replace('全身性', '1').replace('局域性', '2').replace('弥漫性', '3').replace('单侧', '1').replace('双侧', '2').replace('左侧', '3').replace('右侧', '4').replace('左上', '1').replace('左下', '2').replace('右上', '3').replace('右下', '4')

In [ ]:
for i in data:
    # data[i]=data[i].astype('int64')
    if '持续时间' in i:
        data[i]=data[i].astype('int64')
    if '持续时间' not in i:
        data[i]=data[i].astype('category')
    if i == '年龄':
        data[i]=data[i].astype('int64')

数据集分割

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from collections import Counter

In [ ]:
# Separate the features from the labels into 2 objects, X and y
X = data.drop('诊断名称',axis=1)
Y = data['诊断名称']

In [ ]:
# Perform a train test split on the data, with the test size of 10% and a random_state of 202
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# 使用index带出数据集的索引
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=202)

对数据进行normalize

In [ ]:
#Creating a StandardScaler object to normalize the X train and test set feature data
scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)
print(scaled_X_train)

In [ ]:
# 导入LightGBM模型,训练速度更快gbdt
from lightgbm.sklearn import LGBMClassifier

# 定义 LightGBM 模型 
log_model = LGBMClassifier(objective='multiclass',num_class=10,verbose=0,force_col_wise=True)

# 在训练集上训练LightGBM模型
log_model.fit(scaled_X_train, y_train)

In [ ]:
# 获取并打印模型的所有参数
params = log_model.get_params()
for param, value in params.items():
    print(f"{param}: {value}")

In [ ]:
# Model Performance Evaluation

# import evaluation metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.metrics import roc_curve, auc

In [ ]:
#predictions
y_pred = log_model.predict(scaled_X_test)

#confusion matrix
confusion_matrix(y_test,y_pred)

1、TOP1的结果

In [ ]:
#classification report
print(classification_report(y_test,y_pred,digits=3))

对训练集群进行数据平衡

In [ ]:
from collections import Counter
from sklearn.datasets import make_classification
from matplotlib import pyplot
from numpy import where
import joblib as jl
from imblearn.over_sampling import SMOTE

In [ ]:
x = X.values.tolist()
x = numpy.array(x)
y = []
for i in Y:
    y.append(i)
y = numpy.array(y)

In [ ]:
# 确保 X 和 Y 都是数值类型
x = pd.DataFrame(x).apply(pd.to_numeric, errors='coerce')
y = pd.Series(y).apply(pd.to_numeric, errors='coerce')

# 将数据转换为 numpy 数组
x = X.values
y = Y.values

In [ ]:
oversample = SMOTE()
x, y = oversample.fit_resample(x, y)
counter = Counter(y)
counter

In [ ]:
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(x, y, test_size=0.2, random_state=202)

In [ ]:
scaled_X_train_1 = scaler.fit_transform(X_train_1)
scaled_X_test_1 = scaler.transform(X_test_1)

In [ ]:
log_model.fit(scaled_X_train_1, y_train_1)

In [ ]:
y_pred_1 = log_model.predict(scaled_X_test_1)

In [ ]:
#confusion matrix
confusion_matrix(y_test_1,y_pred_1)

In [ ]:
#classification report
print(classification_report(y_test_1,y_pred_1,digits=3))

# 用平衡之后数据集的模型去诊断测试集的数据

In [ ]:
# 读取所有的json文件名
file_path_test = '../test_data/'

FilePath_list_test = []
for root, dirs, files in os.walk(file_path_test):
    for file in files:
        FilePath    = os.path.join(root, file)
        FilePath_list_test.append(FilePath)
print(FilePath_list_test[0])

In [ ]:
disease_symptom_clinical_all_test = []
disease_Inpatient_number_all_test = []
for file_name in FilePath_list_test:
    with open(file_name,'r',encoding='utf-8') as load_f:
        term_ann_list =  []
        disease_name = file_name.split('test_data/')[1].split('_')[0]
        Inpatient_number = file_name.split('test_data/')[1].split('_')[1].split('.json')[0]
        disease_Inpatient_number = disease_name + '_' + Inpatient_number
        disease_Inpatient_number_all_test.append(disease_Inpatient_number)
        for key_d,value_d in disease_classification_dict.items():
            if disease_name in value_d:
                # 假设这些临床特征每个疾病都具备
                # 后面对没有出现的特征进行特换就好了
                load_dict = json.load(load_f)
                annot_infos = load_dict['标注信息']
                for annot_info in annot_infos.values():
                    term_ann_type = annot_info[0]['术语类型']
                    for ssu_attribute in ssu_attributes:
                        if term_ann_type in ['症状','疾病','影像学表现'] and annot_info[1]['存在情况']['标准取值'] != '否认':
                            term_ann_notlab = annot_info[0]['术语名称']
                            # term_loca = annot_info[1]['发作部位']
                            term_attr = annot_info[1][ssu_attribute]['标准取值']
                            for key_s,value_s in term_synonym_dict.items():
                                if term_ann_notlab in value_s:
                                    term_ann_list.append([key_s+'_'+ssu_attribute,term_attr])


                        if term_ann_type == '定量定性检查':
                            term_ann_lab =  annot_info[0]['术语名称'] + annot_info[1]['结果判读']['标准取值']
                            term_attr_lab = annot_info[1][ssu_attribute]['标准取值']
                            for key_s,value_s in term_synonym_dict.items():
                                if term_ann_lab in value_s:
                                    term_ann_list.append([key_s+'_'+ssu_attribute,term_attr_lab])

                        if term_ann_type in ['药物','手术'] and annot_info[1]['存在情况']['标准取值'] != '否认':
                            term_ann_treat =  annot_info[0]['术语名称']
                            term_treat_attr = annot_info[1]['存在情况']['标准取值']
                            for key_s,value_s in term_synonym_dict.items():
                                if term_ann_treat in value_s:
                                    term_ann_list.append([key_s+'_'+'存在情况',term_treat_attr])


                # 因为会有相同症状的不同属性取值，这里只选择出现第一次的,这里没有重复值可以写入字典
                list_fir_symptom = []
                term_ann_first_dict = {}
                for i in range(len(term_ann_list)):
                    if term_ann_list[i][0] not in list_fir_symptom:
                        # print(term_ann_list[i])
                        list_fir_symptom.append(term_ann_list[i][0])
                        term_ann_first_dict[term_ann_list[i][0]] = term_ann_list[i][1]
                # print(term_ann_first_dict)

                res = []
                for x in disease_symptom_attr_list:
                    if x in term_ann_first_dict.keys():
                        res.append(term_ann_first_dict[x])
                    else:
                        res.append('NULL')
                disease_symptom_clinical_all_test.append([key_d,res])


In [ ]:
patient_age_test = []
patient_gender_test = []
for file_name in FilePath_list_test:
    with open(file_name,'r',encoding='utf-8') as load_f:
        load_dict = json.load(load_f)
        patient_age_gender =load_dict['原文信息'].split('。')[0].split('岁,')

        patient_age_test.append(int(patient_age_gender[0]))
        patient_gender_test.append(patient_age_gender[1].replace('男','1').replace('女','0'))

In [ ]:
# 构建完整的dataframe
df_list_test = []
disease_name_test = []
disease_name_1_test = []
for disease_symptom_list_ in disease_symptom_clinical_all_test:
    df_list_test.append(disease_symptom_list_[1])
    disease_name_ = disease_symptom_list_[0]
    # 疾病名称增加到最后一列
    disease_name_1.append(disease_symptom_list_[0])
    disease_name_test.append(disease_name_)
    # 住院号增加到后面
# 将这些症状生成dataframe的形式矩阵
data_test = DataFrame(df_list_test, columns=disease_symptom_attr_list)

In [ ]:
#插入“疾病”“年龄”和“性别”特征
data_test['诊断名称'] = disease_name_test
data_test.insert(loc=0, column='性别', value=patient_gender_test)
data_test.insert(loc=1, column='年龄', value=patient_age_test)

In [ ]:
# 将时间标准化到天
ori_duration_test = []
std_duration_test = []
for column in data_test:
    if '持续时间' in column:
        for duration in data_test[column]:
            if duration != 'NULL':
                if '年' in duration:
                    std_dura = int(duration.rstrip('年'))*365
                if '月' in duration:
                    std_dura = int(duration.rstrip('月'))*30
                if '周' in duration:
                    std_dura = int(duration.rstrip('周'))*7
                if '天' in duration:
                    std_dura = int(duration.rstrip('天'))
                if '小时' in duration:
                    std_dura = int(duration.rstrip('小时'))/24
                if '分钟' in duration:
                    std_dura = int(duration.rstrip('分钟'))/1440
                ori_duration_test.append(duration)
                std_duration_test.append(std_dura)

In [ ]:
# 构建一个需要替换持续时间的字典
std_duration_dict_test = {}
std_duration_dict_test = dict(zip(ori_duration_test, std_duration_test))

In [ ]:
# 把所有的持续时间全部标准化
data_test.replace(std_duration_dict_test,inplace=True)

In [ ]:
data_test = data_test.replace('存在', '1').replace('否认', '0').replace('NULL', '0').replace('既往', '2').replace('可能', '4').replace('假设', '5').replace('家族', '6').replace('轻度', 1).replace('中度', 2).replace('重度', 3).replace('危重', 4).replace('牵拉样痛', '1').replace('针刺样痛', '2').replace('烧灼样痛', '3').replace('钝痛', '4').replace('绞痛', '5').replace('压榨样痛', '6').replace('痉挛样痛', '6').replace('急性', '1').replace('亚急性', '1.5').replace('慢性', '2').replace('持续性', '3').replace('反复发作', '4').replace('周期性', '5').replace('间歇性', '6').replace('一过性', '7').replace('迁延不愈', '8').replace('转移性', '9').replace('晨间发作', '10').replace('夜间发作', '11').replace('午后发作', '12').replace('偶然发作', '13').replace('疾病复发', '14').replace('休息后缓解', '1').replace('活动后加剧', '2').replace('全身性', '1').replace('局域性', '2').replace('弥漫性', '3').replace('单侧', '1').replace('双侧', '2').replace('左侧', '3').replace('右侧', '4').replace('左上', '1').replace('左下', '2').replace('右上', '3').replace('右下', '4')

In [ ]:
# Separate the features from the labels into 2 objects, X and y
X_test_2 = data_test.drop('诊断名称',axis=1)
Y_test_2 = data_test['诊断名称']

In [ ]:
#Creating a StandardScaler object to normalize the X train and test set feature data

scaler = StandardScaler()
scaled_X_train_1 = scaler.fit_transform(X_train_1)
scaled_X_test_2 = scaler.transform(X_test_2)

In [ ]:
y_pred_prob = log_model.predict_proba(scaled_X_test_2)
for y_prob in y_pred_prob:
    diagnose_class = log_model.classes_
    y_prob_top3 = y_prob.argsort()[-3:][::-1]

In [ ]:
y_pred_test = log_model.predict(scaled_X_test_2)

#confusion matrix
confusion_matrix(Y_test_2,y_pred_test)

In [ ]:
#classification report
print(classification_report(Y_test_2,y_pred_test,digits=3))

top3计算，用predict_proba计算出所有疾病的概率可能性，选择prob最大的前三诊断

In [ ]:
y_pred_prob = log_model.predict_proba(scaled_X_test_2)
y_pred_top3_all = []
for y_prob in y_pred_prob:
    diagnose_class = log_model.classes_
    y_prob_top3 = y_prob.argsort()[-3:][::-1]
    y_pred_top3_all.append([diagnose_class[y_prob_top3[0]],diagnose_class[y_prob_top3[1]],diagnose_class[y_prob_top3[2]]])
    # print(diagnose_class[y_prob_top3[0]],diagnose_class[y_prob_top3[1]],diagnose_class[y_prob_top3[2]])
y_pred_top3_all[0]

In [ ]:
ypred_top3 = []
for i in range(len(Y_test_2)):
    if Y_test_2[i] in y_pred_top3_all[i]:
        ypred_top3.append(Y_test_2[i])
    else:
        ypred_top3.append(y_pred_test[i])

In [ ]:
#classification report
print(classification_report(Y_test_2,ypred_top3,digits=3))